# SPDE simulation on a sphere

<!-- SUMMARY: Simulations performed on a Sphere, treated in the SPDE formalism -->

<!-- CATEGORY: SPDE -->

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import gstlearn.plot3D as gop
import numpy as np 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import IPython
from IPython.display import Markdown

gdoc.setNoScroll()

In [ ]:
Markdown(gdoc.loadDoc("SPDE_Sphere.md"))

In [ ]:
# Markov (if False, Matérn covariance will be used)
Markov = True

### Parametrization

In [ ]:
#Sphere radius
R = gl.EARTH_RADIUS
gl.defineDefaultSpace(gl.ESpaceType.SN,param=R)

#Scale parameter (for convenience, it is defined as the proportion of the radius)
ratioRange = 0.2
scale = R * ratioRange

# sill 
sill = 2. 

# Smoothness parameter (for Matérn case)
nu = 2

# Markov coefficients (for Markov case)
coeffs = [1,-1,.5]

### Meshing

In [ ]:
mesh = gl.MeshSphericalExt()
err = mesh.resetFromDb(None,None,triswitch = "-r5",verbose=False)

Sampling Db creation

In [ ]:
nsample = 4000
#sub-sampling to reduce computational burden
np.random.seed(123)
ind = np.random.choice(mesh.getNApices(),size=nsample,replace=False)

#Creation of the db
X = mesh.getCoordinates(0)
Y = mesh.getCoordinates(1)

dbdat = gl.Db.create()
dbdat["x"] = np.array(X)[ind]
dbdat["y"] = np.array(Y)[ind]
dbdat.setLocators(["x","y"],gl.ELoc.X)

### Covariance model

The covariance model is Markov or Matérn.

In [ ]:
if Markov : 
    model = gl.Model.createFromParam(type=gl.ECov.MARKOV,
                                 range = scale,
                                 sill = sill,
                                 flagRange= False)
    model.getCova(0).setMarkovCoeffs(coeffs)
    
else :

    model = gl.Model.createFromParam(type=gl.ECov.BESSEL_K,
                                 range = scale,
                                 sill = sill,
                                 param=nu,
                                 flagRange= False)

### Precision matrix

In [ ]:
Q = gl.PrecisionOp(mesh,model)

### Simulation 

In [ ]:
result = np.array(Q.simulate()[0])

### Display the realization
 

In [ ]:
surface = gop.SurfaceOnMesh(mesh, result,opacity=1)
fig = go.Figure(data=[ surface ])
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
f = fig.show()

### Compute covariance (of discretized solution)

We use the fact that $\Sigma = Q^{-1}$ and solve $Qx = e_j$ for an arbitrary index $j$.

**Get the distances**

In [ ]:
ind0 = 12
distances = np.array(mesh.getDistances(ind0))

**Compute the covariances**

In [ ]:
covDiscr = np.array(Q.evalCov(ind0))

**Sort for the plot**

In [ ]:
covDiscrClose = covDiscr[np.argsort(distances)]
deltaLong =  np.sort(distances)

**Display the result**

In [ ]:
plt.plot(deltaLong,covDiscrClose,"--",label = "Discretized covariance")
ax = plt.legend()

### Variogram of the realization

The empirical variogram is computed by using the great-circle distance.

In [ ]:
npas = 50 # number of discretization points
dpas = 0.04 # lag with respect to the unit sphere (it will be multiplied
# by R in the creation of the VarioParam.

dbdat["simu"] = np.array(result)[ind]
dbdat.setLocators(["simu"],gl.ELoc.Z)

#Variogram 

vp = gl.VarioParam.createOmniDirection(npas=npas,dpas=dpas * R)
vario = gl.Vario.create(vp)
ax = vario.compute(dbdat,gl.ECalcVario.VARIOGRAM)
#vario.display()

In [ ]:
ax = vario.plot(label = "Empirical Variogram")
ax = plt.legend()

### Theoretical covariance function

In [ ]:
Markdown(gdoc.loadDoc("Covariance_Sphere.md"))

### Evaluation 

In [ ]:
ndisc = 100 # number of discretization steps for the covariance 
N = 20 # size of the decomposition

h = np.linspace(0,np.max(deltaLong),ndisc)
ax = vario.plot(label = "Empirical Variogram")
a = model.getCova(0)
uu = np.array([a.evalCovOnSphere(i,N) for i in h]) # modif dR
ax = plt.plot(h, sill - uu,label = "Theoretical Variogram")
plt.plot(deltaLong,covDiscrClose[0] - covDiscrClose,"--",label = "Discretized model")
ax = plt.legend()

There is a slight difference between the theoretical variogram and the one obtained from the SPDE discretization due to a numerical error on the variance introduced by the discretization. The comparison of the covariance shows that this numerical error is rather small :

In [ ]:
h = np.linspace(0,np.max(deltaLong),ndisc)
vario = gl.Vario.create(vp)
ax = vario.compute(dbdat,gl.ECalcVario.COVARIANCE)
#ax = gp.variogram(vario,label = "Empirical Covariance")
ax = plt.plot(h, uu,label = "Theoretical Covariance")
plt.plot(deltaLong,covDiscrClose,"--",label = "Discretized model")
ax = plt.legend()
plt.show()

In [ ]:
h = np.linspace(0,np.max(deltaLong),ndisc)
#vario = gl.Vario.create(vp,dbdat)
#ax = vario.compute(gl.ECalcVario.COVARIANCE)
#ax = gp.variogram(vario,label = "Empirical Covariance")
#ax = plt.plot(h, uu,label = "Theoretical Covariance")
plt.plot(deltaLong,covDiscrClose,"--",label = "Discretized model")
ax = plt.legend()
plt.show()

Kriging